In [1]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [2]:
# Path to the data txt file on disk.
data_path = "Train.csv"
# open the file eng-spa.txt and read
lines= pd.read_csv(data_path,  names =['ID', 'Yoruba', 'English'])
#printing sample data from lines
lines.sample(6)

,ID,Yoruba,English
1053,ID_FLGLgZgs,"--- Àṣìṣe aṣọ́lé ikọ̀ Ac Milan, Gianluigi ṣokù...",--- AC Milan goalkeeper Gianluigi's blunder ma...
9805,ID_yuPnhzkw,Ohun tí mo fún ọ kì í ń ṣe májèlé bikòṣe ẹbu a...,What I gave you is not poison but just mere po...
8132,ID_pwwxJNkT,Ìjàpá ní ọjọ́ tí òún ti jágbọ́n-ọn òo lọrùn ò ...,Tortoise says that since the day it learned th...
3764,ID_TKNRjwgs,"Lóòótọ́, ibùdó méjì tó tóbi déwọ̀n àyè kan wà ...",Although there are two sizeable refugee camps ...
7804,ID_oNGuQmdo,"Ní ìgbà kan rí, Siasia fi Obi wé agbábọ́ọ̀lù ọ...",Siasia had once said Obi was a player at par w...
1982,ID_KBBToZUH,Ó ní Ọriĺẹ̀-èdè yìí ti gbaradì fún eré Ìdárayá...,"Nigeria has put forward the cities, Benin City..."


In [3]:
# convert source and target text to Lowercase 
lines.Yoruba=lines.Yoruba.apply(lambda x: x.lower())
lines.English=lines.English.apply(lambda x: x.lower())
# Remove quotes from source and target text
lines.Yoruba=lines.Yoruba.apply(lambda x: re.sub("'", '', x))
lines.English=lines.English.apply(lambda x: re.sub("'", '', x))
# create a set of all special characters
special_characters= set(string.punctuation)
# Remove all the special characters
lines.Yoruba = lines.Yoruba.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
lines.English = lines.English.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))
# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
lines.Yoruba=lines.Yoruba.apply(lambda x: x.translate(num_digits))
lines.English= lines.English.apply(lambda x: x.translate(num_digits))
# Remove extra spaces
lines.Yoruba=lines.Yoruba.apply(lambda x: x.strip())
lines.English=lines.English.apply(lambda x: x.strip())
lines.Yoruba=lines.Yoruba.apply(lambda x: re.sub(" +", " ", x))
lines.English=lines.English.apply(lambda x: re.sub(" +", " ", x))


In [4]:
# Add start and end tokens to target sequences
lines.English = lines.English.apply(lambda x : 'START_ '+ x + ' _END')
lines.sample(6)

,ID,Yoruba,English
9437,ID_wlQvjodT,nítorí tani ni mo ṣe fẹ́ ṣe ìdáààbòbò,START_ who do i want to protect it from _END
2368,ID_MBNcyCKh,ó da ṣùgbọ́n ó ṣì nílò láti sọ̀rọ̀,START_ oh well but he has to speak up _END
2101,ID_KfEeFJYn,kò sí àwọn ìwífún kan nínú ohun tí o ti sọ níp...,START_ some information is missing from your p...
2791,ID_OIDnYcDH,ojúmọ́ kì í mọ́ kí ọwọ́ má yùnún ẹnu,START_ never a day dawns that the hand does no...
4358,ID_WQxktaHN,bí ó bá jẹ́ pé bó ṣe ṣẹlẹ̀ náà nìyẹn,START_ if that is how it happened _END
2574,ID_NFTMLMbk,àwíìgbọ́ àfọ̀ọ̀gbọ́ tí ń fi àjèjé ọwọ́ mumi,START_ hewhowillnotlistentotalk hewhowillnotli...


In [5]:
# Find all the source and target words and sort them
# Vocabulary of Source language
all_source_words=set()
for source in lines.Yoruba:
    for word in source.split():
        if word not in all_source_words:
            all_source_words.add(word)
# Vocabulary of Target 
all_target_words=set()
for target in lines.English:
    for word in target.split():
        if word not in all_target_words:
            all_target_words.add(word)
# sort all unique source and target words
source_words= sorted(list(all_source_words))
target_words=sorted(list(all_target_words))

In [6]:
#Find maximum sentence length in  the source and target data
source_length_list=[]
for l in lines.Yoruba:
    source_length_list.append(len(l.split(' ')))
max_source_length= max(source_length_list)
print(" Max length of the source sentence",max_source_length)
target_length_list=[]
for l in lines.English:
    target_length_list.append(len(l.split(' ')))
max_target_length= max(target_length_list)
print(" Max length of the target sentence",max_target_length)

 Max length of the source sentence 218
 Max length of the target sentence 160


In [7]:
# creating a word to index(word2idx) for source and target
source_word2idx= dict([(word, i+1) for i,word in enumerate(source_words)])
target_word2idx=dict([(word, i+1) for i, word in enumerate(target_words)])

In [8]:
#creating a dictionary for index to word for source and target vocabulary
source_idx2word= dict([(i, word) for word, i in  source_word2idx.items()])
print(source_idx2word)
target_idx2word =dict([(i, word) for word, i in target_word2idx.items()])

{1: 'a', 2: 'aabo', 3: 'aadun', 4: 'aago', 5: 'aaron', 6: 'aarẹ', 7: 'ababa', 8: 'abaco', 9: 'abaji', 10: 'abala', 11: 'abalaẹ̀kọ́', 12: 'abanijẹ́', 13: 'abare', 14: 'abaribe', 15: 'abayéjẹ́', 16: 'abba', 17: 'abbaaji', 18: 'abbah', 19: 'abbas', 20: 'abbl', 21: 'abbo', 22: 'abdelfattah', 23: 'abdu', 24: 'abdul', 25: 'abdulahi', 26: 'abdulahilorreta', 27: 'abdulfatai', 28: 'abdulkareem', 29: 'abdullahi', 30: 'abdulmumuni', 31: 'abdulrahman', 32: 'abdulrasaq', 33: 'abdulrosheed', 34: 'abdulwaheed', 35: 'abe', 36: 'abel', 37: 'abetíwéré', 38: 'abhay', 39: 'abia', 40: 'abidún', 41: 'abidúnun', 42: 'abilov', 43: 'abily', 44: 'abilékọ', 45: 'abiodun', 46: 'abiola', 47: 'abiyamọ', 48: 'abiyamọ́', 49: 'abiṣuwọ̀rọ̀wọ̀rọ̀lóko', 50: 'abiṣuútabíigi', 51: 'abiṣuútabíòdòdó', 52: 'abo', 53: 'aboyún', 54: 'abreu', 55: 'abu', 56: 'abubakar', 57: 'abuh', 58: 'abuja', 59: 'abujaìpàdé', 60: 'abuké', 61: 'aburú', 62: 'abàgbàdotàkìtàkìlẹ́gàn”', 63: 'abàlèmẹ́fà', 64: 'abàmì', 65: 'abá', 66: 'abájọ', 67: 'abá

In [9]:
#Shuffle the data
lines = shuffle(lines[:3000])

In [10]:
# Train - Test Split
X, y = lines.Yoruba, lines.English
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((2700,), (300,))

In [11]:
# Input tokens for encoder
num_encoder_tokens=len(source_words)
# Input tokens for decoder zero padded
num_decoder_tokens=len(target_words) +1

In [12]:
max_encoder_seq_length=max([len(txt) for txt in source_words])
max_decoder_seq_length=max([len(txt) for txt in target_words])

In [13]:
def generate_batch(X = X_train, y = y_train, batch_size = 50):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_source_length),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_target_length),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_target_length, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                  encoder_input_data[i, t] = source_word2idx[word] 
                  for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_word2idx[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        #print(word)
                        decoder_target_data[i, t - 1, target_word2idx[word]] = 1.
                    
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [14]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 60
epochs = 10
latent_dim=256

In [15]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [16]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [17]:
# Define the model that takes encoder and decoder input 
# to output decoder_outputs
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [18]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [19]:
train_samples = len(X_train) # Total Training samples
val_samples = len(X_test)    # Total validation or test samples
batch_size = 60
epochs = 10

In [20]:

model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
32/45 [====================>.........] - ETA: 1:58 - loss: 0.8577 - acc: 0.0730

InvalidArgumentError: ignored

In [ ]:
model.save_weights('nmt_weights_100epochs.h5')

In [ ]:
model.load_weights('nmt_weights_100epochs.h5')

In [ ]:
# Encode the input sequence to get the "Context vectors"
encoder_model = Model(encoder_inputs, encoder_states)
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_input = [decoder_state_input_h, decoder_state_input_c]
# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs)
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_state_input)
decoder_states2 = [state_h2, state_c2]
# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2)
# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_state_input,
    [decoder_outputs2] + decoder_states2)


In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of 
    #target sequence with the start character.
    target_seq[0, 0] = target_word2idx['START_']
# Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
# Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word =target_idx2word[sampled_token_index]
        decoded_sentence += ' '+ sampled_word
# Exit condition: either hit max length
        # or find stop character.
        if (sampled_word == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True
# Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index
# Update states
        states_value = [h, c]
    return decoded_sentence


In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_train[k:k+1].values[0])
print('Actual Target Translation:', y_train[k:k+1].values[0])
print('Predicted Target Translation:', decoded_sentence)

In [ ]:
test_gen = generate_batch(X_test, y_test, batch_size = 1)
k=10
k+=1
(input_seq, actual_output), _ = next(test_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Source sentence:', X_test[k:k+1].values[0])
print('Actual Target Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Target Translation:', decoded_sentence[:-4])